In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import csv


options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome( options=options)

# 파파고 웹사이트 열기
papago_url = "https://papago.naver.com/"
driver.get(papago_url)

# 페이지 로딩 시간 대기
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "textarea#txtSource")))

list_read = []

file_path = r'C:/Users/user/Desktop/hansol/3rd_project/NLP_project/crawled_data1.csv'

with open(file_path, "r", encoding='utf-8') as f:
    rdr = csv.reader(f)
    list_read = list(rdr)

translated_list = []

wait_t = 30

def translate_text(text):
    input_box = driver.find_element(By.CSS_SELECTOR, "textarea#txtSource")
    input_box.clear()
    input_box.send_keys(text)
    time.sleep(20)
    
    translate_button = driver.find_element(By.CSS_SELECTOR, "button#btnTranslate")
    translate_button.click()
    
    # 번역 결과 대기
    output_element = WebDriverWait(driver, wait_t).until(EC.presence_of_element_located((By.CSS_SELECTOR, "div#txtTarget")))
    
    # 번역 결과 추출
    return output_element.text

# 데이터 번역 및 출력
for ch in list_read:
    if len(ch[0]) >= 3000:
        dot_index = ch[0][3000:2500:-1].index('.')
        x1, x2 = ch[0][:3000-dot_index+1], ch[0][3000-dot_index+1:]
        translated_x1 = translate_text(x1)
        translated_x2 = translate_text(x2)
        sentence = translated_x1 + translated_x2
        translated_list.append(sentence)
    else:
        translated_list.append(translate_text(ch[0]))

driver.quit()

# 결과를 파일에 저장
output_file_path = r'C:/Users/user/Desktop/hansol/3rd_project/NLP_project/completed.csv'
with open(output_file_path, "w", encoding='utf-8', newline='') as f:
    writer = csv.writer(f)
    for sentence in translated_list:
        writer.writerow([sentence])

print(f"{output_file_path} 저장 완료")